<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OnDemandLoaderTool

Our `OnDemandLoaderTool` is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.

This tool takes in a `BaseReader` data loader, and when called will 1) load data, 2) index data, and 3) query the data.

In this walkthrough, we show how to use the `OnDemandLoaderTool` to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

In [ ]:
%pip install llama-index-readers-wikipedia -q

In [ ]:
!pip install llama-index -q

In [4]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

In [ ]:
!pip install wikipedia -q

In [6]:
reader = WikipediaReader()

In [7]:
tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

In [ ]:
!pip install langchain -q

In [ ]:
#added by Frank Morales(FM) 22/02/2024
%pip install openai  --root-user-action=ignore -q
%pip install colab-env --upgrade --quiet --root-user-action=ignore -q

In [ ]:
#added by Frank Morales(FM) 22/02/2024
import warnings
warnings.filterwarnings('ignore')

import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI()

In [ ]:
tool(["Montreal"], query_str="where is located the Montreal city?")

In [30]:
# run tool as langchain structured tool
lc_tool = tool.to_langchain_structured_tool(verbose=True)

In [67]:
from langchain.agents import AgentType, initialize_agent, load_tools
#from langchain_openai import ChatOpenAI, OpenAI
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

#llm = ChatOpenAI(temperature=0.0)
math_llm = OpenAI(temperature=0.0)

#llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)
llm = ChatOpenAI(temperature=0, model_name="gpt-4", streaming=True)

tools = load_tools(
    ["human", "llm-math"],
    llm=math_llm,
)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    #agent="structured-chat-zero-shot-react-description",
    verbose=True,
)

agent = initialize_agent(
    [lc_tool],
    #tools,
    llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=True,
)

In [52]:
agent_chain.run("What's my friend Rafael's surname?")
# Answer with 'Reinaldo'



> Entering new AgentExecutor chain...
I need to ask a human for this information.
Action: human
Action Input: What is Rafael's surname?

What is Rafael's surname?
Reinaldo

Observation: Reinaldo
Thought:I now know Rafael's surname.
Final Answer: Rafael's surname is Reinaldo.

> Finished chain.


"Rafael's surname is Reinaldo."

In [69]:
#agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"},query_str="What's the arts and culture scene in montreal?")

agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"})




> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Wikipedia Tool",
  "action_input": {
    "pages": ["Montreal"],
    "query_str": "arts and culture"
  }
}
```
Observation: Montreal has a rich and vibrant arts and culture scene, with a unique blend of French and English traditions. The city is known for its cultural diversity and has a strong presence in visual arts, theatre, dance, and music. Montreal hosts over 100 annual festivals, including the renowned Montreal International Jazz Festival and the largest comedy festival in the world, Just for Laughs. The city's downtown area comes alive during the summer with various cultural and social events. Additionally, Montreal is recognized for its classical art scene, with venues like Place des Arts hosting world-class orchestras, opera, ballet, and avant-garde dance troupes. The city's churches, including notable basilicas, contribute to its cultural heritage. Quebec literature has also been influenced by Montreal, with w

"Montreal has a rich and vibrant arts and culture scene, with a unique blend of French and English traditions. The city is known for its cultural diversity and has a strong presence in visual arts, theatre, dance, and music. Montreal hosts over 100 annual festivals, including the renowned Montreal International Jazz Festival and the largest comedy festival in the world, Just for Laughs. The city's downtown area comes alive during the summer with various cultural and social events. Additionally, Montreal is recognized for its classical art scene, with venues like Place des Arts hosting world-class orchestras, opera, ballet, and avant-garde dance troupes. The city's churches, including notable basilicas, contribute to its cultural heritage. Quebec literature has also been influenced by Montreal, with works shifting from rural settings to urban portrayals, capturing the city's multicultural essence."

In [68]:
agent.run(input={"pages": "Montreal", "query_str": "What is the best restaurant in Montreal?"})



> Entering new AgentExecutor chain...
Thought: The user is asking for the best restaurant in Montreal. I will use the Wikipedia Tool to find information about Montreal's top restaurants.
Action:
```
{
  "action": "Wikipedia Tool",
  "action_input": {
    "pages": ["Montreal"],
    "query_str": "best restaurant"
  }
}
```
Observation: The best restaurant in Montreal is a subjective matter and can vary depending on personal preferences. Montreal is known for its diverse culinary scene, offering a wide range of dining options from traditional French cuisine to innovative fusion dishes. Visitors and locals alike often recommend exploring the city's vibrant neighborhoods like Old Montreal, Plateau-Mont-Royal, and Mile End to discover hidden gems and popular dining spots. It is advisable to check online reviews, ask locals for recommendations, and explore different types of cuisine to find the best restaurant that suits your taste preferences.
Thought:The Wikipedia tool provided a general 

"The best restaurant in Montreal can vary depending on personal preferences. Montreal is known for its diverse culinary scene, offering a wide range of dining options from traditional French cuisine to innovative fusion dishes. It's recommended to explore the city's vibrant neighborhoods like Old Montreal, Plateau-Mont-Royal, and Mile End to discover popular dining spots. Checking online reviews and asking locals for recommendations can also help you find a restaurant that suits your taste preferences."